!earthengine authenticate

In [1]:
import geopandas as gpd
import ee
import geemap
import numpy as np
import rasterio

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
# from treeinterpreter import treeinterpreter as ti

In [2]:
ee.Initialize()

In [5]:
#shp path
tp_crop ='crop_py.shp'
tp_non_crop = 'non_crop_py.shp'
#read shp as vec
tv_crop = gpd.read_file(tp_crop)
tv_non_crop = gpd.read_file(tp_non_crop)
#cinvert to feature collection
crop_ft = geemap.shp_to_ee(tp_crop)
non_crop_ft = geemap.shp_to_ee(tp_non_crop)
bbox = tv_crop.total_bounds #generate boarding box

traning_d = crop_ft.merge(non_crop_ft)
traning_d
classes = np.unique(traning_d.name)
# classes

class_dict = dict(zip(classes, range(len(classes))))
print(classes)
# tv_non_crop.head()

[<function FeatureCollection.name at 0x000001445042E1F0>]


In [11]:
aoi = ee.Geometry.Rectangle(tv_crop.total_bounds.tolist())

cloudmask = 20
band_sl = ('B2', 'B3', 'B4', 'B8', 'B11', 'B12', 'ndvi')

sentinel2 = ee.ImageCollection("COPERNICUS/S2")\
    .filterBounds(aoi)\
        .filterDate('2020-04-01', '2020-08-31')\
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloudmask))\
                .filter(ee.Filter.lt('CLOUD_COVERAGE_ASSESSMENT',cloudmask))\
                    .select(band_sl)


s_mosaic = sentinel2.median().rename(band_sl)
s_mosaic.getInfo()

trainingSample = s_mosaic.sampleRegions(collection=traning_d, properties=['name'], scale = 10, geometries =True)
trainingSample.size().getInfo()



15971

In [19]:
sentinel2 = ee.ImageCollection("COPERNICUS/S2")\
    .filterBounds(aoi)\
        .filterDate('2020-04-01', '2020-08-31')

In [22]:
ndvi = sentinel2.normalizedDifference(['B8', 'B4']).rename('ndvi')
sentinel2 = s_mosaic.addBands(ndvi)

AttributeError: 'ImageCollection' object has no attribute 'normalizedDifference'

In [14]:
bandname = s_mosaic.bandNames().getInfo()

classifier = ee.Classifier.randonForest(25,0).train(trainingSample)

['B2', 'B3', 'B4', 'B8', 'B11', 'B12']


In [17]:
classifier = ee.Classifier.randomForest(25,0).setOutputMode('PROBABILITY').train(trainingSample,"land_class",bandNames);
classification = image.classify(classifier).multiply(100).clip(cambodia)
# Map = geemap.Map(center=(0.9613,34.6536),zoom=8)
# Map

In [16]:
# Map.addLayer(aoi, name='aoi')
M# ap.addLayer(Ps_mosaic name='Pmosaa)